<a href="https://colab.research.google.com/github/yousri-meftah/Counting_Poeple/blob/main/People_Counting_by_Neuralearn_ai_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
import scipy.io
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape,LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy,custom_loss
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

In [ ]:
import os
from PIL import Image

In [ ]:
IN_X,IN_Y=768,1024
OUT_X,OUT_Y=96,128
SUBSAMPLING_FACTOR=IN_X//OUT_X

# DATA PREPARATION

In [ ]:
def gauss_distribution(x,u=0,sigma=10):
    return np.expand_dims(1/(np.sqrt(2*np.pi*(sigma**2)))*np.exp(-(0.5)*(((x-u)/sigma)**2)),axis=0)

In [ ]:
def get_density_map_gaussian(im,points,gaussian_radius=4):
    density_map=np.zeros((OUT_X,OUT_Y))
    w,h=OUT_Y,OUT_X
    num_gt=len(points)

    for point in points:
        point=np.round(point).astype(int)
        point[0],point[1]=min(h-1,point[1]),min(w-1,point[0])
        x=np.linspace(-gaussian_radius,gaussian_radius,(gaussian_radius*2)+1)
        gaussian_map=np.multiply(gauss_distribution(x),gauss_distribution(x).T)
        gaussian_map/=np.sum(gaussian_map)

        x_left,x_right,y_up,y_down=0,gaussian_map.shape[1],0,gaussian_map.shape[0]
        if point[1]<gaussian_radius:
            x_left=gaussian_radius-point[1]
        if point[0]<gaussian_radius:
            y_up=gaussian_radius-point[0]
        if point[1]+gaussian_radius>=w:
            x_right=gaussian_map.shape[1]-(gaussian_radius+point[1]-w)-1
        if point[0]+gaussian_radius>=h:
            y_down=gaussian_map.shape[0]-(gaussian_radius+point[0]-h)-1
        density_map[
            max(0,point[0]-gaussian_radius):min(density_map.shape[0],point[0]+gaussian_radius+1),
            max(0,point[1]-gaussian_radius):min(density_map.shape[1],point[1]+gaussian_radius+1),
        ]+=gaussian_map[y_up:y_down,x_left:x_right]
    density_map/=np.sum(density_map/len(points))
    return density_map

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__ (self, images, maps, batch_size,SUBSAMPLING_FACTOR=8):

        self.images = images
        self.maps = maps
        self.batch_size = batch_size
        self.train_image_list=os.listdir(images)
        self.SUBSAMPLING_FACTOR=SUBSAMPLING_FACTOR

    def __len__(self):
        return int(np.floor(len(self.train_image_list)/self.batch_size))

    def __getitem__(self, idx):
        x,y = self.__data_generation(idx)
        return x,y

    def __data_generation(self, idx):
        x = []
        y = []

        for j in range(idx*self.batch_size, (idx+1)*self.batch_size):

            im_array=img_to_array(load_img(self.images+os.listdir(self.images)[j],target_size=(IN_X,IN_Y)))
            im_array/=255.
            im_array[:,:,0]=(im_array[:,:,0]-np.mean(im_array[:,:,0]))/np.std(im_array[:,:,0])
            im_array[:,:,1]=(im_array[:,:,1]-np.mean(im_array[:,:,1]))/np.std(im_array[:,:,1])
            im_array[:,:,2]=(im_array[:,:,2]-np.mean(im_array[:,:,2]))/np.std(im_array[:,:,2])
            x.append(im_array)
            mat=scipy.io.loadmat(self.maps+os.listdir(self.maps)[j])
            points=mat['image_info'][0][0][0][0][0]
            points/=self.SUBSAMPLING_FACTOR

            density_map_present=get_density_map_gaussian(im_array,points,gaussian_radius=5)
            y.append(density_map_present)
        return tf.convert_to_tensor(x),tf.convert_to_tensor(y)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [23]:
train_images='/content/dataset/'
train_maps='/content/test_data/'
val_images='//content//ShanghaiTech//part_B//train_data//images/'
val_maps='//content//ShanghaiTech//part_B//train_data/ground-truth/'

LR=1e-4
BATCH_SIZE=1
EPOCH=100
INPUT_DIM=IN_X

In [ ]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.gif')):  # Add more image extensions if needed
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)
            images.append(img_array)
    return images

In [ ]:
images=load_images_from_folder(train_images)
maps=load_images_from_folder(train_maps)

In [24]:
train_gen = DataGenerator(train_images, train_maps,BATCH_SIZE,SUBSAMPLING_FACTOR)
#val_gen = DataGenerator(val_images, val_maps,BATCH_SIZE,INPUT_DIM)

# MODELING

In [17]:
def get_base_model2():
    base_model = VGG16(
        weights='imagenet',
        input_shape=(IN_X, IN_Y, 3),
        include_top=False,
    )

    block4_conv3 = base_model.get_layer("block4_conv3").output
    return Model(
        inputs=base_model.inputs, outputs=block4_conv3
    )
get_base_model2().summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 768, 1024, 3)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 768, 1024, 64)     1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 768, 1024, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 384, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 384, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 384, 512, 128)     147584    
                                                              

In [18]:
def get_base_model():
    base_model = VGG16(
        weights='imagenet',
        input_shape=(IN_X,IN_Y,3),
        include_top=False,)

    block4_conv3=[base_model.get_layer(layer_name).output for layer_name in ["block4_conv3"]]
    return Model(
        inputs=[base_model.inputs],outputs=[block4_conv3]
    )

get_base_model().summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 768, 1024, 3)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 768, 1024, 64)     1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 768, 1024, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 384, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 384, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 384, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 192, 256, 128)     0   

In [37]:
def get_base_model(input_shape):
    base_model = VGG16(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    base_model_output = base_model.get_layer("block4_conv3").output
    return Model(inputs=base_model.input, outputs=base_model_output)

#IN_X, IN_Y = [your_dimensions]  # Replace with actual dimensions
base_model = get_base_model((IN_X, IN_Y, 3))

inputs = Input(shape=(IN_X, IN_Y, 3))
x = base_model(inputs)
init = RandomNormal(0.01)
base_model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 768, 1024, 3)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 768, 1024, 64)     1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 768, 1024, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 384, 512, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 384, 512, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 384, 512, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 192, 256, 128)     0   

In [38]:


x=Conv2D(512, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(512, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(512, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(256, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(128, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(64, (3,3), activation = 'relu', dilation_rate=2,kernel_initializer=init,padding='same')(x)
x=Conv2D(1, (1,1), activation = 'relu', dilation_rate=1,kernel_initializer=init,padding='same')(x)


#================

#================
out=Reshape((OUT_X,OUT_Y))(x)
model=tf.keras.Model(inputs=inputs,outputs=out)
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 768, 1024, 3)]    0         
                                                                 
 model_7 (Functional)        (None, 96, 128, 512)      7635264   
                                                                 
 conv2d_28 (Conv2D)          (None, 96, 128, 512)      2359808   
                                                                 
 conv2d_29 (Conv2D)          (None, 96, 128, 512)      2359808   
                                                                 
 conv2d_30 (Conv2D)          (None, 96, 128, 512)      2359808   
                                                                 
 conv2d_31 (Conv2D)          (None, 96, 128, 256)      1179904   
                                                                 
 conv2d_32 (Conv2D)          (None, 96, 128, 128)      2950

In [39]:
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = Adam(learning_rate = LR),
    metrics='accuracy',
    run_eagerly = True,
)

In [ ]:
checkpoint_filepath='...'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True
)

In [27]:
# Suppose we have some fake data and labels
fake_logits = np.random.randn(96, 128)  # This would be the output of your model

# Labels should be integers between 0 and 127 because we have 128 classes
fake_labels = np.random.randint(0, 128, size=(96,))

# Now we can calculate the sparse categorical crossentropy
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss = loss_fn(fake_labels, fake_logits)

print(loss.numpy())


5.1437342502895484


In [40]:
history = model.fit(
    train_gen,
    verbose=1,
    shuffle=True,
    epochs=EPOCH
)

Epoch 1/100
  5/300 [..............................] - ETA: 4:22:12 - loss: 7998039780229120.0000 - accuracy: 0.0104

KeyboardInterrupt: ignored

# TESTING

In [ ]:
IN ='...'

im_array=img_to_array(load_img(train_image+IN_'.jpg',target_size=(IN_X,IN_Y)))
im_array/=255.
im_array[:,:,0]=(im_array[:,:,0]-np.mean(im_array[:,:,0]))/np.std(im_array[:,:,0])
im_array[:,:,1]=(im_array[:,:,1]-np.mean(im_array[:,:,1]))/np.std(im_array[:,:,1])
im_array[:,:,2]=(im_array[:,:,2]-np.mean(im_array[:,:,2]))/np.std(im_array[:,:,2])

plt.figure(figsize=(20,12))
plt.imhow(im_array)

output=mode.predict(tf.expand_dims(im_array,axis=0))
output=np.reshape(output,(OUT_X,OUT_Y))

n_people=np.sum(output)
mat=scipy.io.loadmat(train_maps+'GT_'+IN+'.mat')
points=mat['image_info'][0][0][0][0][0]
points/=SUBSAMPLING_FACTOR

num_gt=np.squeeze(points).shape[0]
print("The actual number of people is=",num_gt)
print("The predicted number of people is =",n_people)

plt.figure(figsize=(20,12))
plt.imshow(output)